In [23]:
import gc
import torch
import evaluate
import transformers
import numpy as np
import pandas as pd

from tqdm import tqdm
from datetime import datetime
from transformers import AutoTokenizer
from transformers import GenerationConfig
from transformers import AutoModelForCausalLM
from evaluate.visualization import radar_plot
from nltk.tokenize import sent_tokenize

In [24]:
dir_root = "Data/"
log_prefix = "log_test_history_"
max_token = 256 # max token from dataset
min_token = 92 # min token from dataset
type = pd.read_json(f"{dir_root}qaa_test.json", orient="records")["input"].values[0]

In [3]:
def calc_cossin(ref, pred):
    st = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
    
    sentences_ref = sent_tokenize(ref)
    encode_ref = st.encode(sentences_ref)
    
    sentences_pred = sent_tokenize(pred)
    encode_pred = st.encode(sentences_pred)
    
    cosine_scores = util.cos_sim(encode_pred, encode_ref).numpy()
    
    return sum([item.max() for item in cosine_scores]) / len(cosine_scores)

In [4]:
def get_best_model(prefix, metric, data):
    columns = [prefix + str(round(treshold,1)) + metric for treshold in np.arange(1.0, -0.1, -0.1)]
    describe = data[columns].describe().T
    val = describe.max()["mean"]
    name = describe[describe["mean"] == val].index[0]

    return name, val

In [5]:
def measure(col, samples):
    cossins = []
    rouge1 = []
    rouge2 = []
    rougeL = []
    rougeLsum = []
    meteors = []
    bleus = []
    bleu_precisions_n1 = []
    bleu_precisions_n2 = []
    bleu_precisions_n3 = []
    bleu_precisions_n4 = []
    bleu_brevity_penalty = []
    bleu_length_ratio = []
    bleu_translation_length = []
    bleu_reference_length = []
    
    rouge = evaluate.load('rouge')
    meteor = evaluate.load('meteor')
    bleu = evaluate.load("bleu")
    st = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
    
    for i in tqdm(range(len(samples))):
        # Cosine sim
        cossins.append(calc_cossin(samples["output"].values[i], test_topic[col].values[i]))
        
        # rouge
        r_rouge = rouge.compute(
            predictions=[samples[col].values[i]],
            references=[samples["output"].values[i]]
        )
        rouge1.append(r_rouge["rouge1"])
        rouge2.append(r_rouge["rouge2"])
        rougeL.append(r_rouge["rougeL"])
        rougeLsum.append(r_rouge["rougeLsum"])

        # meteor
        r_meteor = meteor.compute(predictions=[samples[col].values[i]], references=[samples["output"].values[i]])
        meteors.append(r_meteor["meteor"])

        # bleu
        try:
            r_bleu = bleu.compute(predictions=[samples[col].values[i]], references=[samples["output"].values[i]])
            bleus.append(r_bleu["bleu"])
            bleu_precisions_n1.append(r_bleu["precisions"][0])
            bleu_precisions_n2.append(r_bleu["precisions"][1])
            bleu_precisions_n3.append(r_bleu["precisions"][2])
            bleu_precisions_n4.append(r_bleu["precisions"][3])
            bleu_brevity_penalty.append(r_bleu["brevity_penalty"])
            bleu_length_ratio.append(r_bleu["length_ratio"])
            bleu_translation_length.append(r_bleu["translation_length"])
            bleu_reference_length.append(r_bleu["reference_length"])
        except:
            bleus.append(0.0)
            bleu_precisions_n1.append(0.0)
            bleu_precisions_n2.append(0.0)
            bleu_precisions_n3.append(0.0)
            bleu_precisions_n4.append(0.0)
            bleu_brevity_penalty.append(0.0)
            bleu_length_ratio.append(0.0)
            bleu_translation_length.append(0.0)
            bleu_reference_length.append(0.0)

    samples[f"{col}_cs"] = cossins
    samples[f"{col}_rouge1"] = rouge1
    samples[f"{col}_rouge2"] = rouge2
    samples[f"{col}_rougeL"] = rougeL
    samples[f"{col}_rougeLsum"] = rougeLsum
    samples[f"{col}_meteor"] = meteors
    samples[f"{col}_bleu"] = bleus
    samples[f"{col}_bleu_precisions_n1"] = bleu_precisions_n1
    samples[f"{col}_bleu_precisions_n2"] = bleu_precisions_n2
    samples[f"{col}_bleu_precisions_n3"] = bleu_precisions_n3
    samples[f"{col}_bleu_precisions_n4"] = bleu_precisions_n4
    samples[f"{col}_bleu_brevity_penalty"] = bleu_brevity_penalty
    samples[f"{col}_bleu_length_ratio"] = bleu_length_ratio
    samples[f"{col}_bleu_translation_length"] = bleu_translation_length
    samples[f"{col}_bleu_reference_length"] = bleu_reference_length

In [6]:
def generate(prompt, tokenizer, model):
    inputs = tokenizer(
        prompt,
        return_tensors="pt",
    )# .to(device)
    input_ids = inputs["input_ids"].cuda()    

    generation_config = GenerationConfig(
        temperature=0.6,
        top_p=0.95,
        do_sample=True,
        repetition_penalty=1.2,
    )

    # print("Generating...")
    generation_output = model.generate(
        input_ids=input_ids,
        generation_config=generation_config,
        return_dict_in_generate=True,
        output_scores=True,
        min_new_tokens=min_token,
        max_new_tokens=max_token,
        pad_token_id = 0,
        # eos_token_id = 50256
        eos_token_id = tokenizer.eos_token_id 
    )

    result = []
    for s in generation_output.sequences:
        result.append(tokenizer.decode(s))

    return result

In [7]:
def run(samples, tokenizer, model):
    results = []
    for i in tqdm(range(0, len(samples))):
        prompt = f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
{samples["instruction"].values[i]}
### Input:
{samples["input"].values[i]}
### Response:"""
    
        result = generate(prompt, tokenizer, model)
        results.append(result[0].split("### Response:\n")[-1])
        
    return results

In [8]:
import torch
import transformers
from sentence_transformers import  util
from sentence_transformers import SentenceTransformer

class DomainExpert():
    def __init__(
        self, 
        top_k=10, 
        min_new_tokens=0,
        max_new_tokens=512,         
        num_return_sequences=1,
        temperature=0.9,
        top_p=0.6,
        repetition_penalty=1.2,
        do_sample=True,
        max_input=4000,
        model_st_name="sentence-transformers/all-MiniLM-L6-v2",
        model_chat_name="meta-llama/Llama-2-7b-chat-hf",
        devices="auto"):
            
        self.top_k = top_k
        self.min_new_tokens = min_new_tokens
        self.max_new_tokens = max_new_tokens
        self.num_return_sequences = num_return_sequences
        self.temperature = temperature
        self.top_p = top_p
        self.repetition_penalty=repetition_penalty
        self.do_sample = do_sample
        self.model_st_name = model_st_name
        self.model_chat_name = model_chat_name
        self.devices = devices
        self.max_input = 4000
                
        self.model_st = SentenceTransformer(self.model_st_name)

        self.pipeline_chat = transformers.pipeline(
            "text-generation",
            model=self.model_chat_name,
            torch_dtype=torch.float16,
            device_map=self.devices,
        )

        self.tokenizer_chat = transformers.AutoTokenizer.from_pretrained(self.model_chat_name)

    def get_number_of_tokens(self, context):
        return len(self.tokenizer_chat(context)["input_ids"])
    
    def sentence_encode(self, sentence):
        return self.model_st.encode([sentence])

    def mask_prompt(self, question, context = ""):
        return f"""[INST]Below is an instruction that describes a task! Write a response that appropriately completes the request. To answer the instruction, use the context if available!
Instruction: {question}
Context: {context}[/INST]"""
        
    def create_prompt(self, question, context):
        """
        Megadja a promptot, hogy mi alapján válaszoljon a modell.
        """
        prompt = ""
        context_tmp = ""
        if len(context) > 0:
            for text in context:            
                context_tmp += f"{text}\n"
                prompt_tmp = self.mask_prompt(question, context_tmp)
                if len(self.tokenizer_chat(prompt_tmp)["input_ids"]) < self.max_input:
                    prompt = prompt_tmp
                else:
                    break
        else:
            prompt = self.mask_prompt(question)
        
        return prompt
    
    def generate_answare(self, question, context):
        """
        Vissza adja a kérdés és a kontextus alapján a választ!
        """
        prompt = self.create_prompt(question, context)
        # print(question)
        # print(context)
        # print(prompt)
        sequences = self.pipeline_chat(
            prompt,
            do_sample = self.do_sample,
            top_k = self.top_k,
            num_return_sequences = self.num_return_sequences,
            min_new_tokens = self.min_new_tokens,
            max_new_tokens = self.max_new_tokens,
            temperature = self.temperature,
            top_p = self.top_p,
            repetition_penalty = self.repetition_penalty,
            eos_token_id = self.tokenizer_chat.eos_token_id
        )

        answ = sequences[0]['generated_text'].split("[/INST]")[1][2:].split("\n")
        answ = "\n".join([item for item in answ if len(item) > 0])
        # print(answ)        
        return answ

In [9]:
import torch
import transformers
from sentence_transformers import  util
from sentence_transformers import SentenceTransformer

class DomainExpertV2():
    def __init__(
        self, 
        top_k=10, 
        min_new_tokens=0,
        max_new_tokens=512,         
        num_return_sequences=1,
        temperature=0.9,
        top_p=0.6,
        repetition_penalty=1.2,
        do_sample=True,
        max_input=4096,
        model_st_name="sentence-transformers/all-MiniLM-L6-v2",
        model_chat_name="meta-llama/Llama-2-7b-chat-hf",
        tokenizer_name="meta-llama/Llama-2-7b-chat-hf",
        type="Domain Expert",
        devices="auto"):
            
        self.top_k = top_k
        self.min_new_tokens = min_new_tokens
        self.max_new_tokens = max_new_tokens
        self.num_return_sequences = num_return_sequences
        self.temperature = temperature
        self.top_p = top_p
        self.repetition_penalty=repetition_penalty
        self.do_sample = do_sample
        self.model_st_name = model_st_name
        self.model_chat_name = model_chat_name
        self.tokenizer_name = tokenizer_name
        self.type = type
        self.devices = devices
        self.max_input = max_input - self.max_new_tokens
                
        self.model_st = SentenceTransformer(self.model_st_name)
        self.tokenizer_chat = transformers.AutoTokenizer.from_pretrained(self.tokenizer_name, 
                                                                         padding=True, 
                                                                         truncation=True) 
            
        self.pipeline_chat = transformers.pipeline(
            "text-generation",
            model=self.model_chat_name,
            tokenizer=self.tokenizer_chat,
            torch_dtype=torch.float16,
            device_map=self.devices,
        )

    def get_number_of_tokens(self, context):
        return len(self.tokenizer_chat(context)["input_ids"])
    
    def sentence_encode(self, sentence):
        return self.model_st.encode([sentence])

    def mask_promt(self, question, context = ""):
        return f"""[INST]Below is an instruction that describes a task! Write a response that appropriately completes the request. To answer the instruction, use the context if available!
### Instruction: 
{question}

### Input:
{self.type}

### Context: 
{context}
[/INST]

### Response:
"""
        
    def create_prompt(self, question, context):
        """
        Megadja a promptot, hogy mi alapján válaszoljon a modell.
        """
        prompt = ""
        context_tmp = ""
        if len(context) > 0:
            for text in context:            
                context_tmp += f"{text}\n"
                # prompt_tmp = f"""[INST]Summarize and explain the given context in term of given question and start your answare below way 'Based on the information available to me': 
                # Question: {question}
                # Context: {context_tmp}[/INST]"""    
                # prompt_tmp = f"""[INST]Summarize and explain the given context in terms of the given question and start your answer below way 'Based on the information available to me':
                # Question: {question}
                # Context: {context_tmp}[/INST]"""
                # prompt_tmp = f"""[INST]Answer the given question based on the given context.
                # Question: {question}
                # Context: {context_tmp}[/INST]"""
                prompt_tmp = self.mask_promt(question, context_tmp)
                if len(self.tokenizer_chat(prompt_tmp)["input_ids"]) < self.max_input:
                    prompt = prompt_tmp
                else:
                    break
        else:
            prompt = self.mask_promt(question)
                
        return prompt
    
    def generate_answare(self, question, context):
        """
        Vissza adja a kérdés és a kontextus alapján a választ!
        """
        prompt = self.create_prompt(question, context)
        # print("question:", question)
        # print("context:", context)        
        # print("prompt len", len(self.tokenizer_chat(prompt)["input_ids"]))
        # print("prompt:", prompt)
        
        sequences = self.pipeline_chat(
            prompt,
            do_sample = self.do_sample,
            top_k = self.top_k,
            num_return_sequences = self.num_return_sequences,
            min_new_tokens = self.min_new_tokens,
            max_new_tokens = self.max_new_tokens,
            temperature = self.temperature,
            top_p = self.top_p,
            repetition_penalty = self.repetition_penalty,
            eos_token_id = self.tokenizer_chat.eos_token_id,
            pad_token_id = self.tokenizer_chat.eos_token_id
        )

        # print(sequences)
        answ = sequences[0]['generated_text'].split("[/INST]")[1]
        answ = answ.replace("### Response:\n","")
        answ = answ.split("\n")
        answ = "\n".join([item for item in answ if len(item) > 0])
        # print("ANSWER\n", answ)
        # except:
        #     answ = ""
        # print(answ)
        return answ

In [10]:
import torch
import transformers
from sentence_transformers import  util
from sentence_transformers import SentenceTransformer

class DomainExpertV3():
    def __init__(
        self, 
        top_k=10, 
        min_new_tokens=0,
        max_new_tokens=512,         
        num_return_sequences=1,
        temperature=0.9,
        top_p=0.6,
        repetition_penalty=1.2,
        do_sample=True,
        max_input=4000,
        model_st_name="sentence-transformers/all-MiniLM-L6-v2",
        model_chat_name="meta-llama/Llama-2-7b-chat-hf",
        devices="auto"):
            
        self.top_k = top_k
        self.min_new_tokens = min_new_tokens
        self.max_new_tokens = max_new_tokens
        self.num_return_sequences = num_return_sequences
        self.temperature = temperature
        self.top_p = top_p
        self.repetition_penalty=repetition_penalty
        self.do_sample = do_sample
        self.model_st_name = model_st_name
        self.model_chat_name = model_chat_name
        self.devices = devices
        self.max_input = 4000
                
        self.model_st = SentenceTransformer(self.model_st_name)        
        self.tokenizer_chat = transformers.AutoTokenizer.from_pretrained(self.model_chat_name)                
        
        self.pipeline_chat = transformers.pipeline(
            "text-generation",
            model=self.model_chat_name,
            torch_dtype=torch.float16,
            device_map=self.devices,
        )        

    def get_number_of_tokens(self, context):
        return len(self.tokenizer_chat(context)["input_ids"])
    
    def sentence_encode(self, sentence):
        return self.model_st.encode([sentence])

    def mask_prompt(self, question, context = "", support_model = ""):
        return f"""[INST]Below is an instruction that describes a task! Write a response that appropriately completes the request. Use the context and additional information to answer the prompt!
Instruction: {question}
Context: {support_model}
Additional information: {context}[/INST]"""
        
    def create_prompt(self, question, context, support_model):
        """
        Megadja a promptot, hogy mi alapján válaszoljon a modell.
        """
        prompt = ""
        context_tmp = ""
        if len(context) > 0 and len(support_model) > 0:
            for text in context:            
                context_tmp += f"{text}\n"
                prompt_tmp = self.mask_prompt(question=question, 
                                              context=context_tmp, 
                                              support_model=support_model)
                if len(self.tokenizer_chat(prompt_tmp)["input_ids"]) < self.max_input:
                    prompt = prompt_tmp
                else:
                    break
        else:
            prompt = self.mask_prompt(question=question, 
                                      support_model=support_model)
        
        return prompt
    
    def generate_answare(self, question, context, support_model):
        """
        Vissza adja a kérdés és a kontextus alapján a választ!
        """
        prompt = self.create_prompt(question, context, support_model) 
        sequences = self.pipeline_chat(
            prompt,
            do_sample = self.do_sample,
            top_k = self.top_k,
            num_return_sequences = self.num_return_sequences,
            min_new_tokens = self.min_new_tokens,
            max_new_tokens = self.max_new_tokens,
            temperature = self.temperature,
            top_p = self.top_p,
            repetition_penalty = self.repetition_penalty,
            eos_token_id = self.tokenizer_chat.eos_token_id
        )

        answ = sequences[0]['generated_text'].split("[/INST]")[1][2:].split("\n")
        answ = "\n".join([item for item in answ if len(item) > 0])    
        return answ

In [11]:
import json
import numpy as np
import pandas as pd

from sentence_transformers import  util

class ExtractorCOVID():
    def __init__(self, path, domain_expert):
        self.path = path
        self.domain_expert = domain_expert
        
        self.meta = pd.read_csv(f"{self.path}/metadata.csv", low_memory=False)
        self.meta = self.meta[~self.meta["title"].isnull()]
        self.meta = self.meta[~self.meta["abstract"].isnull()]

        self.embedded_vectors = np.load(f"{self.path}titles.emb")

        with open(f"{self.path}titles.idx","r") as f:
            self.index_to_vectors = f.read().splitlines()
        self.index_to_vectors = [index for index in self.index_to_vectors]

    def get_realted_cosine_scores(self):
        if len(self.selected_indexes) > 0 and len(self.cosine_scores):
            return self.cosine_scores[self.selected_indexes]
            
    def get_context(self, question=None, treshold=0.75, N=100):
        # calculate similarities
        self.embedded_question = self.domain_expert.sentence_encode(question)
        self.cosine_scores = util.cos_sim(self.embedded_question, self.embedded_vectors)
        self.cosine_scores = self.cosine_scores.reshape(len(self.cosine_scores[0])).numpy()
    
        # Sort similarities
        self.selected_indexes = np.argsort(self.cosine_scores)
        # Filter limit similarities
        self.selected_indexes = self.selected_indexes[::-1][:N]              
        # Filter treshold
        self.selected_indexes = [self.selected_indexes[i] for i in range(len(self.cosine_scores[self.selected_indexes])) if self.cosine_scores[self.selected_indexes[i]] >= treshold]
        # self.cosine_scores = np.argsort(self.cosine_scores)

        # find selected elements
        self.selected_elements = [self.index_to_vectors[si] for si in self.selected_indexes]        
        self.selected_references = self.meta[self.meta["cord_uid"].isin(self.selected_elements)][["publish_time", "authors", "pdf_json_files", "pmc_json_files"]]
        
        # Create context
        self.title = self.meta[self.meta["cord_uid"].isin(self.selected_elements)]["title"].values
        self.context = self.meta[self.meta["cord_uid"].isin(self.selected_elements)]["abstract"].values
        
        return self.context

In [12]:
import json
import numpy as np
import pandas as pd

from sentence_transformers import  util

class ExtractorBase():
    def __init__(self, path, domain_expert):
        self.path = path
        self.domain_expert = domain_expert
        
        with open(f"{self.path}qaa_q.json") as f:
            self.qaa = json.load(f)

        with open(f"{self.path}qaa_q.idx","r") as f:
            self.index_to_vectors = f.read().splitlines()
        self.index_to_vectors = [int(index) for index in self.index_to_vectors]
            
        self.embedded_vectors = np.load(f"{self.path}qaa_q.emb")

    def get_realted_cosine_scores(self):
        if len(self.selected_indexes) > 0 and len(self.cosine_scores):
            return self.cosine_scores[self.selected_indexes]

    def clear_item(self, item):
        item = item.replace(" .", "")
        item = item.replace("..", ".")
        return item
        
    def get_context(self, question=None, treshold=0.75, N=100):
        # calculate similarities
        self.embedded_question = self.domain_expert.sentence_encode(question)
        self.cosine_scores = util.cos_sim(self.embedded_question, self.embedded_vectors)
        self.cosine_scores = self.cosine_scores.reshape(len(self.cosine_scores[0])).numpy()
    
        # Sort similarities
        self.selected_indexes = np.argsort(self.cosine_scores)
        # Filter limit similarities
        self.selected_indexes = self.selected_indexes[::-1][:N]              
        # Filter treshold
        self.selected_indexes = [self.selected_indexes[i] for i in range(len(self.cosine_scores[self.selected_indexes])) if self.cosine_scores[self.selected_indexes[i]] >= treshold]
        # self.cosine_scores = np.argsort(self.cosine_scores)        

        # Find elements
        self.selected_elements = [self.qaa[self.index_to_vectors[si]] for si in self.selected_indexes]
        
        # Find duplicated elements
        self.duplicated_indexes = pd.DataFrame([se["answer"] for se in self.selected_elements])
        self.duplicated_indexes = self.duplicated_indexes[self.duplicated_indexes.duplicated()]
        if len(self.duplicated_indexes) > 0:
            self.duplicated_indexes = self.duplicated_indexes.groupby(list(self.duplicated_indexes)).apply(lambda x: x.index[0]).tolist()
            self.selected_indexes = [self.selected_indexes[i] for i in range(0, len(self.selected_indexes)) if i not in self.duplicated_indexes]
            self.selected_elements = [self.qaa[self.index_to_vectors[si]] for si in self.selected_indexes]

        # find selected elements        
        self.selected_answares = [se["answer"] for se in self.selected_elements]
        self.selected_answares = pd.DataFrame(self.selected_answares)
        
        # self.selected_references = {
        #     "question_focus" : [se["question_focus"] for se in self.selected_elements],
        #     "question_type" : [se["question_type"] for se in self.selected_elements]
        # }            
        # self.selected_references = pd.DataFrame(self.selected_references).drop_duplicates()
        
        # Create context
        self.context = [self.clear_item(item[0]) for item in self.selected_answares.values]

        return self.context

In [13]:
import json
import numpy as np
import pandas as pd

from sentence_transformers import  util

class ExtractorBaseF():
    def __init__(self, path, domain_expert):
        self.max_return = 200 # It surely is more than 4000 tokens
        
        self.path = path
        self.domain_expert = domain_expert 
        
        self.sentences = pd.read_json(f"{self.path}/sentences.json")
        self.embedded_vectors = np.load(f"{self.path}sentences.emb")

    def get_realted_cosine_scores(self):
        if len(self.selected_indexes) > 0 and len(self.cosine_scores):
            return self.cosine_scores[self.selected_indexes]

    def get_context(self, question=None, treshold=0.75, N=100):
        # calculate similarities
        self.embedded_question = self.domain_expert.sentence_encode(question)
        self.cosine_scores = util.cos_sim(self.embedded_question, self.embedded_vectors)
        self.cosine_scores = self.cosine_scores.reshape(len(self.cosine_scores[0])).numpy()
    
        # Sort similarities
        self.selected_indexes = np.argsort(self.cosine_scores)
        # Filter limit similarities
        self.selected_indexes = self.selected_indexes[::-1][:N]              
        # Filter treshold
        self.selected_indexes = [self.selected_indexes[i] for i in range(len(self.cosine_scores[self.selected_indexes])) if self.cosine_scores[self.selected_indexes[i]] >= treshold]
        # self.cosine_scores = np.argsort(self.cosine_scores)

        self.context = list(self.sentences[self.sentences.index.isin(self.selected_indexes)]["sentences"].values[:200])
        
        self.selected_references = self.sentences[self.sentences.index.isin(self.selected_indexes)]
        self.selected_references = self.selected_references.drop(columns=["sentences"])
        self.selected_references = self.selected_references[:200]
                                                                         
        return self.context

In [14]:
def rag_test(domain_expert, extractor, data, mark, N=100, log_dir="Data/"):    
    for treshold in np.arange(1.0, -0.1, -0.1):
        try:
            treshold = round(treshold,1)
            answares = []
        
            # pbar = tqdm(range(len(data["instruction"].values)))
            for i in range(len(data["instruction"].values)):   
                context = extractor.get_context(data["instruction"].values[i], treshold=treshold, N=100)
                # if len(context) == 0:
                #     answares.append("")
                # else:
                answare = domain_expert.generate_answare(
                    question = data["instruction"].values[i], 
                    context = context)        
                answares.append(answare)
            
                out = f"{datetime.now()} : {treshold} : {len(context)} : {i}"
                print(out)
                with open(f"{log_dir}rag_test_{mark}status.log","a") as f:
                    f.write(out + "\n")
                
            data[mark + str(treshold)] = answares
        except Exception as e:
            data[mark + str(treshold)] = ""
            out = f"{datetime.now()} : {treshold} : {len(context)} : {i}"
            with open(f"{log_dir}rag_test_{mark}status.log","a") as f:
                f.write(str(e) + "\n")
                f.write(out + "\n")

    return data

In [15]:
def rag_test_h(domain_expert, extractor, data, mark, N=100):
    for treshold in np.arange(1.0, -0.1, -0.1):    
        treshold = round(treshold,1)
        # treshold = 0.5
        answares = []
    
        pbar = tqdm(range(len(data["instruction"].values)))
        for i in pbar:   
            context = extractor.get_context(data["instruction"].values[i], treshold=treshold, N=100)
            # if len(context) == 0:
            #     answares.append("")
            # else:
            answare = domain_expert.generate_answare(
                question = data["instruction"].values[i], 
                context = context,
                support_model = data["llama_fn"].values[i]
            )        
            answares.append(answare)
        
            pbar.set_description(f"Treshold: {treshold}, context len: {len(context)}")
    
        data[mark + str(treshold)] = answares

    return data

In [16]:
from transformers import AutoConfig

config_gptj = AutoConfig.from_pretrained('EleutherAI/gpt-j-6B')
print('EleutherAI/gpt-j-6B', config_gptj.max_position_embeddings)

config_opt = AutoConfig.from_pretrained('facebook/opt-6.7b')
print('facebook/opt-6.7b', config_opt.max_position_embeddings)

config_llama = AutoConfig.from_pretrained('huggyllama/llama-7b')
print('huggyllama/llama-7b', config_llama.max_position_embeddings)

config_llama2 = AutoConfig.from_pretrained('meta-llama/Llama-2-7b-hf')
print('meta-llama/Llama-2-7b-hf', config_llama2.max_position_embeddings)

EleutherAI/gpt-j-6B 2048
facebook/opt-6.7b 2048
huggyllama/llama-7b 2048
meta-llama/Llama-2-7b-hf 4096


## Test RAG vs Find-Tune Models

### GPTJ FN

In [20]:
test_topic = pd.read_json(f"{dir_root}qaa_test.json", orient="records")
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-j-6B")
model = AutoModelForCausalLM.from_pretrained("Models/gptj-6b-v20240926/checkpoint-480", device_map="auto")

/opt/conda/envs/domaine/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [21]:
results = run(test_topic, tokenizer, model)
test_topic["gptj_fn"] = results
test_topic.to_json(f"{dir_root}/qaa_test_gptj_fn.json", orient="records")
test_topic.head()

  0% 0/300 [00:00<?, ?it/s]From v4.47 onwards, when a model cache is to be returned, `generate` will return a `Cache` instance instead by default (as opposed to the legacy tuple of tuples format). If you want to keep returning the legacy format, please set `return_legacy_cache=True`.
100% 300/300 [48:01<00:00,  9.61s/it]


,instruction,output,input,gptj_fn
0,What is (are) actin-accumulation myopathy ?,Actin-accumulation myopathy is a disorder that...,MedQuAD Super Expert,"Actin-accumulation myopathy, also called oculo..."
1,Is Hydrocephalus due to congenital stenosis of...,Is hydrocephalus due to congenital stenosis of...,MedQuAD Super Expert,How is hydrocephalus due to congenital stenosi...
2,What are the symptoms of Laing distal myopathy ?,What are the signs and symptoms of Laing dista...,MedQuAD Super Expert,What are the signs and symptoms of Laing dista...
3,How many people are affected by hereditary hyp...,X-linked hypophosphatemic rickets is the most ...,MedQuAD Super Expert,Hereditary hypophosphatemic rickets affects ab...
4,What are the genetic changes related to Li-Fra...,The CHEK2 and TP53 genes are associated with L...,MedQuAD Super Expert,Mutations in at least four genes increase the ...


In [24]:
model = None
tokenizer = None
test_topic = None
gc.collect()
torch.cuda.empty_cache()

### GPTJ RAG with Context FN model

In [42]:
domain_expert = DomainExpertV2(min_new_tokens=min_token, 
                               max_new_tokens=max_token,
                               type=type,
                               max_input=config_gptj.max_position_embeddings,
                               model_chat_name="Models/gptj-6b-context-v20231216/checkpoint-960",
                               tokenizer_name="EleutherAI/gpt-j-6B")

/opt/conda/envs/domaine/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

#### Sentecen based

In [18]:
test_topic_s = pd.read_json(f"{dir_root}qaa_test.json", orient="records")
ebf = ExtractorBaseF(path = dir_root, domain_expert = domain_expert)
print(ebf.embedded_vectors.shape)
test_topic_s = rag_test(domain_expert, ebf, test_topic_s, mark = "gptj_fnc_rag_s_", N=1000)
test_topic_s.to_json(f"{dir_root}/qaa_test_gptj_fnc_s.json", orient="records")
test_topic_s.head()

#### Question based

In [ ]:
test_topic_q = pd.read_json(f"{dir_root}test_topic.json", orient="records")
eb = ExtractorBase(path = dir_root, domain_expert = domain_expert)
print(eb.embedded_vectors.shape)
test_topic_q = rag_test(domain_expert, eb, test_topic_q, "gptjc_rag_")
test_topic_q.to_json(f"{dir_root}/test_topic_gptj_fnc_q.json", orient="records")
test_topic_q.head()

(28790, 384)


  0% 0/279 [00:00<?, ?it/s]/opt/conda/envs/test/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Treshold: 1.0, context len: 0:   0% 1/279 [00:08<40:00,  8.63s/it]/opt/conda/envs/test/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Treshold: 1.0, context len: 0:   1% 2/279 [00:17<40:03,  8.68s/it]/opt/conda/envs/test/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Treshold: 1.0, context len: 0:   1% 3/279 [00:25<39:40,  8.63s/it]/opt/conda/envs/test/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarnin

### OPT FN

In [19]:
test_topic = pd.read_json(f"{dir_root}qaa_test.json", orient="records")
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-6.7b")
model = AutoModelForCausalLM.from_pretrained("Models/opt-6.7b-v20240926/checkpoint-480", device_map="auto")

/opt/conda/envs/domaine/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

In [20]:
results = run(test_topic, tokenizer, model)
test_topic["opt_fn"] = results
test_topic.to_json(f"{dir_root}/qaa_test_opt_fn.json", orient="records")
test_topic.head()

100% 300/300 [36:30<00:00,  7.30s/it]


,instruction,output,input,opt_fn
0,What is (are) actin-accumulation myopathy ?,Actin-accumulation myopathy is a disorder that...,MedQuAD Super Expert,Actin-accumulation myopathy is one of several ...
1,Is Hydrocephalus due to congenital stenosis of...,Is hydrocephalus due to congenital stenosis of...,MedQuAD Super Expert,How is hydrocephalus due to congenital stenosi...
2,What are the symptoms of Laing distal myopathy ?,What are the signs and symptoms of Laing dista...,MedQuAD Super Expert,What are the signs and symptoms of Laing dista...
3,How many people are affected by hereditary hyp...,X-linked hypophosphatemic rickets is the most ...,MedQuAD Super Expert,The prevalence of hereditary hypophosphatemic ...
4,What are the genetic changes related to Li-Fra...,The CHEK2 and TP53 genes are associated with L...,MedQuAD Super Expert,Mutations in the LIFR gene cause Li-Fraumeni s...


In [21]:
model = None
tokenizer = None
test_topic = None
gc.collect()
torch.cuda.empty_cache()

### OPT RAG with Context FN model

In [19]:
domain_expert = DomainExpertV2(min_new_tokens=min_token, 
                               max_new_tokens=max_token,
                               type=type,
                               max_input=config_opt.max_position_embeddings,
                               model_chat_name="Models/opt-6.7b-context-v20231215/checkpoint-2037",
                               tokenizer_name="facebook/opt-6.7b")

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

#### Sentecen based

In [ ]:
test_topic_s = pd.read_json(f"{dir_root}test_topic.json", orient="records")
ebf = ExtractorBaseF(path = dir_root, domain_expert = domain_expert)
print(ebf.embedded_vectors.shape)
test_topic_s = rag_test(domain_expert, ebf, test_topic_s, mark = "optc_rag_s_", N=1000)
test_topic_s.to_json(f"{dir_root}/test_topic_optc_s.json", orient="records")
test_topic_s.head()

(37874, 384)


/opt/conda/envs/test/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


2023-12-20 08:04:58.057627 : 1.0 : 0 : 0


/opt/conda/envs/test/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


2023-12-20 08:05:05.680855 : 1.0 : 0 : 1


...

/opt/conda/envs/test/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


2023-12-20 10:06:53.354799 : 0.7 : 1 : 87


/opt/conda/envs/test/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


2023-12-20 10:07:01.264250 : 0.7 : 0 : 88


/opt/conda/envs/test/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


#### Question based

In [ ]:
test_topic_q = pd.read_json(f"{dir_root}test_topic.json", orient="records")
eb = ExtractorBase(path = dir_root, domain_expert = domain_expert)
print(eb.embedded_vectors.shape)
test_topic_q = rag_test(domain_expert, eb, test_topic_q, "optc_rag_q_")
test_topic_q.to_json(f"{dir_root}/test_topic_optc_q.json", orient="records")
test_topic_q.head()

(28790, 384)


  0% 0/279 [00:00<?, ?it/s]/opt/conda/envs/test/lib/python3.10/site-packages/transformers/generation/utils.py:1518: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
Treshold: 1.0, context len: 0:   4% 10/279 [01:10<31:16,  6.98s/it]/opt/conda/envs/test/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Treshold: 1.0, context len: 0:   4% 11/279 [01:17<31:16,  7.00s/it]/opt/conda/envs/test/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU

### LLama FN

In [22]:
test_topic = pd.read_json(f"{dir_root}qaa_test.json", orient="records")
tokenizer = AutoTokenizer.from_pretrained("huggyllama/llama-7b")
model = AutoModelForCausalLM.from_pretrained("Models/llama-7b-v20240927/checkpoint-480", device_map="auto")

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message.


Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

In [23]:
results = run(test_topic, tokenizer, model)
test_topic["llama_fn"] = results
test_topic.to_json(f"{dir_root}/qaa_test_llama_fn.json", orient="records")
test_topic.head()

  0% 0/300 [00:00<?, ?it/s]Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)
From v4.47 onwards, when a model cache is to be returned, `generate` will return a `Cache` instance instead by default (as opposed to the legacy tuple of tuples format). If you want to keep returning the legacy format, please set `return_legacy_cache=True`.
100% 300/300 [43:15<00:00,  8.65s/it]


,instruction,output,input,llama_fn
0,What is (are) actin-accumulation myopathy ?,Actin-accumulation myopathy is a disorder that...,MedQuAD Super Expert,Actin-accumulation myopathy is characterized b...
1,Is Hydrocephalus due to congenital stenosis of...,Is hydrocephalus due to congenital stenosis of...,MedQuAD Super Expert,How is hydrocephalus due to congenital stenosi...
2,What are the symptoms of Laing distal myopathy ?,What are the signs and symptoms of Laing dista...,MedQuAD Super Expert,What are the signs and symptoms of Laing dista...
3,How many people are affected by hereditary hyp...,X-linked hypophosphatemic rickets is the most ...,MedQuAD Super Expert,The prevalence of hereditary hypophosphatemic ...
4,What are the genetic changes related to Li-Fra...,The CHEK2 and TP53 genes are associated with L...,MedQuAD Super Expert,Mutations in the LFS gene cause Li-Fraumeni sy...


In [24]:
model = None
tokenizer = None
test_topic = None
gc.collect()
torch.cuda.empty_cache()

### LLama RAG with Context FN model

In [ ]:
domain_expert = DomainExpertV2(min_new_tokens=min_token, 
                               max_new_tokens=max_token,
                               type=type,
                               max_input=config_llama.max_position_embeddings,
                               model_chat_name="Models/llama-7b-context-v20231115/checkpoint-2037",
                               tokenizer_name="huggyllama/llama-7b")

#### Sentecen based

In [ ]:
test_topic_s = pd.read_json(f"{dir_root}test_topic.json", orient="records")
ebf = ExtractorBaseF(path = dir_root, domain_expert = domain_expert)
print(ebf.embedded_vectors.shape)
test_topic_s = rag_test(domain_expert, ebf, test_topic_s, mark = "llamac_rag_s_", N=1000)
test_topic_s.to_json(f"{dir_root}/test_topic_llamac_s.json", orient="records")
test_topic_s.head()

#### Question based

In [ ]:
test_topic_q = pd.read_json(f"{dir_root}test_topic.json", orient="records")
eb = ExtractorBase(path = dir_root, domain_expert = domain_expert)
print(eb.embedded_vectors.shape)
test_topic_q = rag_test(domain_expert, eb, test_topic_q, "llamac_rag_q_")
test_topic_q.to_json(f"{dir_root}/test_topic_llamac_q.json", orient="records")
test_topic_q.head()

### LLama2 FN

In [27]:
test_topic = pd.read_json(f"{dir_root}qaa_test.json", orient="records")
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf")
model = AutoModelForCausalLM.from_pretrained("Models/llama-2-7b-hf-v20241001/checkpoint-320", device_map="auto")

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

In [28]:
results = run(test_topic, tokenizer, model)
test_topic["llama2_fn"] = results
test_topic.to_json(f"{dir_root}/qaa_test_llama2_fn.json", orient="records")
test_topic.head()

  0% 0/300 [00:00<?, ?it/s]Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)
From v4.47 onwards, when a model cache is to be returned, `generate` will return a `Cache` instance instead by default (as opposed to the legacy tuple of tuples format). If you want to keep returning the legacy format, please set `return_legacy_cache=True`.
100% 300/300 [43:35<00:00,  8.72s/it]


,instruction,output,input,llama2_fn
0,What is (are) actin-accumulation myopathy ?,Actin-accumulation myopathy is a disorder that...,MedQuAD Super Expert,Actin-accumulation myopathy is a disorder that...
1,Is Hydrocephalus due to congenital stenosis of...,Is hydrocephalus due to congenital stenosis of...,MedQuAD Super Expert,How is hydrocephalus due to congenital stenosi...
2,What are the symptoms of Laing distal myopathy ?,What are the signs and symptoms of Laing dista...,MedQuAD Super Expert,What are the signs and symptoms of Laing dista...
3,How many people are affected by hereditary hyp...,X-linked hypophosphatemic rickets is the most ...,MedQuAD Super Expert,The incidence of hereditary hypophosphatemic r...
4,What are the genetic changes related to Li-Fra...,The CHEK2 and TP53 genes are associated with L...,MedQuAD Super Expert,The TP53 gene has been associated with the dev...


In [29]:
model = None
tokenizer = None
test_topic = None
gc.collect()
torch.cuda.empty_cache()

### LLama2 Base

In [28]:
test_topic = pd.read_json(f"{dir_root}qaa_test.json", orient="records")
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-hf", device_map="auto")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [29]:
results = run(test_topic, tokenizer, model)
test_topic["llama2_base"] = results
test_topic.to_json(f"{dir_root}/qaa_test_llama2_base.json", orient="records")
test_topic.head()

100% 300/300 [35:37<00:00,  7.13s/it]


,instruction,output,input,llama2_base
0,What is (are) actin-accumulation myopathy ?,Actin-accumulation myopathy is a disorder that...,MedQuAD Super Expert,<s> Below is an instruction that describes a t...
1,Is Hydrocephalus due to congenital stenosis of...,Is hydrocephalus due to congenital stenosis of...,MedQuAD Super Expert,The most common cause for hydrocephalus in inf...
2,What are the symptoms of Laing distal myopathy ?,What are the signs and symptoms of Laing dista...,MedQuAD Super Expert,The answer for this question should be in the ...
3,How many people are affected by hereditary hyp...,X-linked hypophosphatemic rickets is the most ...,MedQuAD Super Expert,The number of patients with hereditary hypopho...
4,What are the genetic changes related to Li-Fra...,The CHEK2 and TP53 genes are associated with L...,MedQuAD Super Expert,Li Fraumeni Syndrome (LFS) is caused by mutati...


In [30]:
model = None
tokenizer = None
test_topic = None
gc.collect()
torch.cuda.empty_cache()

### LLama2 RAG with base model

In [31]:
domain_expert = DomainExpert(min_new_tokens=min_token, 
                             max_new_tokens=max_token)

/opt/conda/envs/domaine/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

#### Sentecen based

In [32]:
test_topic = pd.read_json(f"{dir_root}qaa_test.json", orient="records")
ebf = ExtractorBaseF(path = dir_root, domain_expert = domain_expert)
print(ebf.embedded_vectors.shape)
test_topic = rag_test(domain_expert, ebf, test_topic, mark = "llama2_rag_s_", N=1000)
test_topic.to_json(f"{dir_root}/qaa_test_llama2_base_s.json", orient="records")
test_topic.head()

(130777, 384)
2024-09-28 17:41:42.771017 : 1.0 : 0 : 0
2024-09-28 17:41:52.396146 : 1.0 : 1 : 1
2024-09-28 17:42:01.818822 : 1.0 : 0 : 2
2024-09-28 17:42:06.021035 : 1.0 : 0 : 3
2024-09-28 17:42:15.555870 : 1.0 : 0 : 4
2024-09-28 17:42:25.046657 : 1.0 : 0 : 5
2024-09-28 17:42:31.498991 : 1.0 : 0 : 6
2024-09-28 17:42:41.005175 : 1.0 : 0 : 7
2024-09-28 17:42:47.643028 : 1.0 : 0 : 8
2024-09-28 17:42:52.642091 : 1.0 : 0 : 9


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


2024-09-28 17:42:57.140567 : 1.0 : 0 : 10
2024-09-28 17:43:06.736416 : 1.0 : 0 : 11
2024-09-28 17:43:16.217491 : 1.0 : 0 : 12
2024-09-28 17:43:25.703252 : 1.0 : 0 : 13
2024-09-28 17:43:31.065018 : 1.0 : 0 : 14
2024-09-28 17:43:37.924254 : 1.0 : 0 : 15
2024-09-28 17:43:47.430712 : 1.0 : 0 : 16
2024-09-28 17:43:55.704258 : 1.0 : 0 : 17
2024-09-28 17:44:05.040421 : 1.0 : 0 : 18
2024-09-28 17:44:14.113678 : 1.0 : 0 : 19
2024-09-28 17:44:23.562883 : 1.0 : 0 : 20
2024-09-28 17:44:33.096371 : 1.0 : 0 : 21
2024-09-28 17:44:40.676789 : 1.0 : 0 : 22
2024-09-28 17:44:45.235837 : 1.0 : 0 : 23
2024-09-28 17:44:51.445685 : 1.0 : 0 : 24
2024-09-28 17:45:01.022393 : 1.0 : 0 : 25
2024-09-28 17:45:10.584364 : 1.0 : 0 : 26
2024-09-28 17:45:16.699219 : 1.0 : 0 : 27
2024-09-28 17:45:26.223396 : 1.0 : 0 : 28
2024-09-28 17:45:31.087064 : 1.0 : 0 : 29
2024-09-28 17:45:40.564357 : 1.0 : 0 : 30
2024-09-28 17:45:47.949605 : 1.0 : 1 : 31
2024-09-28 17:45:57.499691 : 1.0 : 0 : 32
2024-09-28 17:46:06.999256 : 1.0 :

This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (4096). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.


2024-09-28 20:30:41.420324 : 0.6 : 100 : 22
2024-09-28 20:30:51.278163 : 0.6 : 100 : 23
2024-09-28 20:30:57.382196 : 0.6 : 50 : 24
2024-09-28 20:31:07.184311 : 0.6 : 53 : 25
2024-09-28 20:31:16.897270 : 0.6 : 57 : 26
2024-09-28 20:31:21.606567 : 0.6 : 29 : 27
2024-09-28 20:31:31.702849 : 0.6 : 95 : 28
2024-09-28 20:31:38.404151 : 0.6 : 13 : 29
2024-09-28 20:31:43.593193 : 0.6 : 67 : 30
2024-09-28 20:31:51.412982 : 0.6 : 28 : 31
2024-09-28 20:32:01.039534 : 0.6 : 23 : 32
2024-09-28 20:32:10.765617 : 0.6 : 21 : 33
2024-09-28 20:32:20.300554 : 0.6 : 39 : 34
2024-09-28 20:32:29.931685 : 0.6 : 15 : 35
2024-09-28 20:32:39.941395 : 0.6 : 100 : 36
2024-09-28 20:32:49.998215 : 0.6 : 100 : 37
2024-09-28 20:32:55.378690 : 0.6 : 100 : 38
2024-09-28 20:33:04.950867 : 0.6 : 23 : 39
2024-09-28 20:33:13.657790 : 0.6 : 60 : 40
2024-09-28 20:33:20.581050 : 0.6 : 48 : 41
2024-09-28 20:33:30.147843 : 0.6 : 1 : 42
2024-09-28 20:33:39.824803 : 0.6 : 25 : 43
2024-09-28 20:33:44.135857 : 0.6 : 6 : 44
2024-09-

,instruction,output,input,llama2_rag_s_1.0,llama2_rag_s_0.9,llama2_rag_s_0.8,llama2_rag_s_0.7,llama2_rag_s_0.6,llama2_rag_s_0.5,llama2_rag_s_0.4,llama2_rag_s_0.3,llama2_rag_s_0.2,llama2_rag_s_0.1,llama2_rag_s_0.0
0,What is (are) actin-accumulation myopathy ?,Actin-accumulation myopathy is a disorder that...,MedQuAD Super Expert,Actin-accumulation myopathies are a group of r...,Actin-accumulation myopathies are a group of r...,"Great, thank you for providing the context! Ba...","Based on the provided information, actin-accum...",Actin-accumulation myopathy is a rare genetic ...,Actin-accumulation myopathy is a disorder that...,Actin-accumulation myopathy is a disorder that...,Actin-accumulation myopathy is a disorder that...,Actin-accumulation myopathy is a disorder that...,Actin-accumulation myopathy is a disorder that...,Actin-accumulation myopathy is a disorder that...
1,Is Hydrocephalus due to congenital stenosis of...,Is hydrocephalus due to congenital stenosis of...,MedQuAD Super Expert,"According to medical research and literature, ...","Great, thank you for providing the instruction...","Great, thank you for providing me with the ins...","Great, thank you for providing this informatio...",Based on the information provided in the instr...,"Great, thank you for providing me with the ins...",Great! Based on the provided instructions and ...,Great! Based on the provided instructions and ...,Great! Based on the provided instructions and ...,Great! Based on the provided instructions and ...,Great! Based on the provided instructions and ...
2,What are the symptoms of Laing distal myopathy ?,What are the signs and symptoms of Laing dista...,MedQuAD Super Expert,Thank you for providing the context! Based on ...,"Based on the provided context, the symptoms of...","According to the Human Phenotype Ontology, the...",Based on the provided context and instructions...,Instruction: What are the symptoms of Laing di...,Instruction: What are the symptoms of Laing di...,Instruction: What are the symptoms of Laing di...,Instruction: What are the symptoms of Laing di...,Instruction: What are the symptoms of Laing di...,Instruction: What are the symptoms of Laing di...,Instruction: What are the symptoms of Laing di...
3,How many people are affected by hereditary hyp...,X-linked hypophosphatemic rickets is the most ...,MedQuAD Super Expert,According to the National Organization for Rar...,Thank you for providing the context! According...,"According to the provided information, hypopho...","According to the information provided, it is e...",According to the provided instructions and con...,"According to the provided text, approximately ...","According to the provided text, approximately ...","According to the provided instructions, you we...","According to the provided instructions, you ar...","According to the provided text, approximately ...","According to the provided instructions, you ar..."
4,What are the genetic changes related to Li-Fra...,The CHEK2 and TP53 genes are associated with L...,MedQuAD Super Expert,"Great, thank you for providing the context! Ba...","Great, thank you for providing the context! Ac...",Thank you for providing the context! Based on ...,"Based on the provided context, the genetic cha...",The genetic changes related to Li-Fraumeni syn...,Based on the provided instructions and context...,"Based on the provided instruction and context,...",Based on the provided instructions and context...,"Based on the provided instruction and context,...",Based on the provided instructions and context...,Based on the provided instructions and context...


#### Question based

In [37]:
test_topic = pd.read_json(f"{dir_root}qaa_test.json", orient="records")
eb = ExtractorBase(path = dir_root, domain_expert = domain_expert)
print(eb.embedded_vectors.shape)
test_topic = rag_test(domain_expert, eb, test_topic, "llama2_rag_q_")
test_topic.to_json(f"{dir_root}/qaa_test_llama2_base_q.json", orient="records")
test_topic.head()

(12815, 384)
2024-09-29 07:11:12.033427 : 1.0 : 0 : 0
2024-09-29 07:11:16.892380 : 1.0 : 1 : 1
2024-09-29 07:11:26.296610 : 1.0 : 1 : 2
2024-09-29 07:11:31.239547 : 1.0 : 0 : 3
2024-09-29 07:11:39.456766 : 1.0 : 0 : 4
2024-09-29 07:11:48.829674 : 1.0 : 1 : 5
2024-09-29 07:11:56.374730 : 1.0 : 2 : 6
2024-09-29 07:12:05.828192 : 1.0 : 1 : 7
2024-09-29 07:12:12.505982 : 1.0 : 0 : 8
2024-09-29 07:12:21.889025 : 1.0 : 2 : 9
2024-09-29 07:12:31.296145 : 1.0 : 1 : 10
2024-09-29 07:12:40.752129 : 1.0 : 1 : 11
2024-09-29 07:12:50.256919 : 1.0 : 1 : 12
2024-09-29 07:12:59.678486 : 1.0 : 2 : 13
2024-09-29 07:13:03.282465 : 1.0 : 1 : 14
2024-09-29 07:13:08.126318 : 1.0 : 0 : 15
2024-09-29 07:13:17.589136 : 1.0 : 2 : 16
2024-09-29 07:13:26.910121 : 1.0 : 1 : 17
2024-09-29 07:13:36.240121 : 1.0 : 0 : 18
2024-09-29 07:13:44.823770 : 1.0 : 1 : 19
2024-09-29 07:13:54.114939 : 1.0 : 1 : 20
2024-09-29 07:13:58.455158 : 1.0 : 3 : 21
2024-09-29 07:14:07.094528 : 1.0 : 0 : 22
2024-09-29 07:14:15.007734 : 1.

/tmp/ipykernel_554/2825354160.py:51: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  self.duplicated_indexes = self.duplicated_indexes.groupby(list(self.duplicated_indexes)).apply(lambda x: x.index[0]).tolist()


2024-09-29 07:40:11.292649 : 1.0 : 16 : 222
2024-09-29 07:40:16.855484 : 1.0 : 0 : 223
2024-09-29 07:40:20.958510 : 1.0 : 1 : 224
2024-09-29 07:40:27.277543 : 1.0 : 1 : 225
2024-09-29 07:40:36.719392 : 1.0 : 1 : 226
2024-09-29 07:40:46.127407 : 1.0 : 0 : 227
2024-09-29 07:40:55.838616 : 1.0 : 2 : 228
2024-09-29 07:41:01.789370 : 1.0 : 2 : 229
2024-09-29 07:41:08.712450 : 1.0 : 0 : 230
2024-09-29 07:41:18.064406 : 1.0 : 0 : 231
2024-09-29 07:41:27.468683 : 1.0 : 1 : 232
2024-09-29 07:41:36.894740 : 1.0 : 0 : 233
2024-09-29 07:41:46.251197 : 1.0 : 0 : 234
2024-09-29 07:41:55.734924 : 1.0 : 3 : 235
2024-09-29 07:42:05.251513 : 1.0 : 1 : 236
2024-09-29 07:42:12.806188 : 1.0 : 2 : 237
2024-09-29 07:42:22.227380 : 1.0 : 1 : 238
2024-09-29 07:42:31.592925 : 1.0 : 0 : 239
2024-09-29 07:42:40.978447 : 1.0 : 1 : 240
2024-09-29 07:42:45.719691 : 1.0 : 0 : 241
2024-09-29 07:42:55.133000 : 1.0 : 0 : 242
2024-09-29 07:42:58.827192 : 1.0 : 1 : 243
2024-09-29 07:43:07.712519 : 1.0 : 1 : 244
2024-09-29

/tmp/ipykernel_554/2825354160.py:51: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  self.duplicated_indexes = self.duplicated_indexes.groupby(list(self.duplicated_indexes)).apply(lambda x: x.index[0]).tolist()


2024-09-29 07:48:17.350624 : 1.0 : 1 : 283
2024-09-29 07:48:26.737950 : 1.0 : 0 : 284
2024-09-29 07:48:36.293488 : 1.0 : 1 : 285
2024-09-29 07:48:45.687026 : 1.0 : 2 : 286
2024-09-29 07:48:52.508161 : 1.0 : 0 : 287
2024-09-29 07:48:58.202882 : 1.0 : 0 : 288
2024-09-29 07:49:02.989317 : 1.0 : 1 : 289
2024-09-29 07:49:06.469198 : 1.0 : 1 : 290
2024-09-29 07:49:15.865910 : 1.0 : 0 : 291
2024-09-29 07:49:22.042444 : 1.0 : 1 : 292
2024-09-29 07:49:30.014389 : 1.0 : 0 : 293
2024-09-29 07:49:39.392818 : 1.0 : 0 : 294
2024-09-29 07:49:48.776017 : 1.0 : 0 : 295
2024-09-29 07:49:58.181282 : 1.0 : 2 : 296
2024-09-29 07:50:07.557387 : 1.0 : 0 : 297
2024-09-29 07:50:16.916183 : 1.0 : 0 : 298
2024-09-29 07:50:21.569216 : 1.0 : 1 : 299
2024-09-29 07:50:31.006518 : 0.9 : 1 : 0
2024-09-29 07:50:37.868637 : 0.9 : 2 : 1
2024-09-29 07:50:47.271980 : 0.9 : 1 : 2
2024-09-29 07:50:52.758963 : 0.9 : 1 : 3
2024-09-29 07:50:59.295578 : 0.9 : 1 : 4
2024-09-29 07:51:08.737935 : 0.9 : 1 : 5
2024-09-29 07:51:15.983

...

/tmp/ipykernel_554/2825354160.py:51: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  self.duplicated_indexes = self.duplicated_indexes.groupby(list(self.duplicated_indexes)).apply(lambda x: x.index[0]).tolist()


2024-09-29 14:49:18.094140 : 0.0 : 98 : 297
2024-09-29 14:49:22.182821 : 0.0 : 100 : 298


/tmp/ipykernel_554/2825354160.py:51: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  self.duplicated_indexes = self.duplicated_indexes.groupby(list(self.duplicated_indexes)).apply(lambda x: x.index[0]).tolist()


2024-09-29 14:49:26.420306 : 0.0 : 99 : 299


,instruction,output,input,llama2_rag_q_1.0,llama2_rag_q_0.9,llama2_rag_q_0.8,llama2_rag_q_0.7,llama2_rag_q_0.6,llama2_rag_q_0.5,llama2_rag_q_0.4,llama2_rag_q_0.3,llama2_rag_q_0.2,llama2_rag_q_0.1,llama2_rag_q_0.0
0,What is (are) actin-accumulation myopathy ?,Actin-accumulation myopathy is a disorder that...,MedQuAD Super Expert,Actin-accumulation myopathies are a group of r...,"Based on the provided context, actin-accumulat...","Based on the provided information, actin-accum...","Based on the provided instruction and context,...","According to the provided text, actin-accumula...","According to the provided text, actin-accumula...","According to the provided text, actin-accumula...","Based on the provided instruction, here is the...","According to the provided text, actin-accumula...","According to the provided text, actin-accumula...","According to the provided text, actin-accumula..."
1,Is Hydrocephalus due to congenital stenosis of...,Is hydrocephalus due to congenital stenosis of...,MedQuAD Super Expert,"According to the provided information, hydroce...","According to the provided information, hydroce...","Based on the provided instruction and context,...","Based on the provided instruction and context,...",Based on the provided instructions and context...,"Great! Based on the provided instructions, I h...",Based on the provided instructions and context...,"Based on the provided instructions, I have com...","According to the provided text, hydrocephalus ...","According to the provided text, hydrocephalus ...","Based on the provided instructions, I have com..."
2,What are the symptoms of Laing distal myopathy ?,What are the signs and symptoms of Laing dista...,MedQuAD Super Expert,"According to the provided information, the sym...","Based on the provided context, the symptoms of...",Based on the provided instructions and context...,Based on the provided instructions and the giv...,"According to the provided instructions, I have...",Based on the provided instructions and the giv...,Based on the provided instructions and referen...,Based on the provided instructions and referen...,Based on the provided instructions and referen...,Based on the provided instructions and the giv...,"According to the provided instructions, I have..."
3,How many people are affected by hereditary hyp...,X-linked hypophosphatemic rickets is the most ...,MedQuAD Super Expert,According to the National Organization for Rar...,"According to the provided context, approximate...","According to the provided text, approximately ...","According to the provided text, approximately ...","According to the provided instructions, the an...","According to the instructions provided, the an...","According to the provided instructions, the an...","According to the instructions provided, the an...","According to the provided instructions, the an...","According to the provided instructions, the an...","According to the provided instructions, the an..."
4,What are the genetic changes related to Li-Fra...,The CHEK2 and TP53 genes are associated with L...,MedQuAD Super Expert,"Great, thank you for providing the context! Ba...","Based on the provided context, the genetic cha...","Based on the provided context, the genetic cha...",Instructions related to Li-Fraumeni syndrome:\...,Instructions related to Li-Fraumeni syndrome:\...,Instructions related to Li-Fraumeni syndrome:\...,Instructions related to Li-Fraumeni syndrome:\...,Instructions related to Li-Fraumeni syndrome:\...,Instructions related to Li-Fraumeni syndrome:\...,Instructions: Complete the requested tasks bas...,Instructions related to Li-Fraumeni syndrome:\...


In [38]:
domain_expert.model_st = None
domain_expert.pipeline_chat = None
domain_expert = None
eb = None
ebf = None
test_topic = None
gc.collect()
torch.cuda.empty_cache()

### LLama2 RAG with FN model

In [17]:
domain_expert = DomainExpertV2(min_new_tokens=min_token, 
                               max_new_tokens=max_token,
                               type=type,
                               max_input=config_llama2.max_position_embeddings,
                               model_chat_name="Models/llama-2-7b-hf-v20241001/checkpoint-320",
                               tokenizer_name="meta-llama/Llama-2-7b-hf")

/opt/conda/envs/domaine/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

In [26]:
# test_topic = pd.read_json(f"{dir_root}qaa_test.json", orient="records")
# test_topic.head()

In [25]:
# test_topic_v2 = pd.read_json(f"{dir_root}qaa_test_opt_fn.json", orient="records")
# test_topic_v2 = test_topic_v2[["instruction","output","input"]]
# test_topic_v2.to_json(f"{dir_root}/qaa_test.json", orient="records")
# test_topic_v2.head()

#### Sentecen based

In [19]:
test_topic = pd.read_json(f"{dir_root}qaa_test.json", orient="records")
# print(len(test_topic))
# test_topic = test_topic[:2]
# print(len(test_topic))
ebf = ExtractorBaseF(path = dir_root, domain_expert = domain_expert)
print(ebf.embedded_vectors.shape)
test_topic = rag_test(domain_expert, ebf, test_topic, mark = "llama2_fn_rag_s_", N=1000)
test_topic.to_json(f"{dir_root}/qaa_test_llama2_fn_s.json", orient="records")
test_topic.head()

300
300
(130777, 384)
2024-10-01 13:31:48.573371 : 1.0 : 0 : 0
2024-10-01 13:31:58.207593 : 1.0 : 1 : 1
2024-10-01 13:32:07.809646 : 1.0 : 0 : 2
2024-10-01 13:32:17.339160 : 1.0 : 0 : 3
2024-10-01 13:32:26.972071 : 1.0 : 0 : 4


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


2024-10-01 13:32:36.561764 : 1.0 : 0 : 5
2024-10-01 13:32:46.121475 : 1.0 : 0 : 6
2024-10-01 13:32:55.657499 : 1.0 : 0 : 7
2024-10-01 13:33:05.236808 : 1.0 : 0 : 8
2024-10-01 13:33:14.874991 : 1.0 : 0 : 9
2024-10-01 13:33:24.437629 : 1.0 : 0 : 10
2024-10-01 13:33:34.079149 : 1.0 : 0 : 11
2024-10-01 13:33:43.637486 : 1.0 : 0 : 12
2024-10-01 13:33:53.219032 : 1.0 : 0 : 13
2024-10-01 13:34:02.746675 : 1.0 : 0 : 14
2024-10-01 13:34:12.366591 : 1.0 : 0 : 15
2024-10-01 13:34:21.864192 : 1.0 : 0 : 16
2024-10-01 13:34:31.469260 : 1.0 : 0 : 17
2024-10-01 13:34:41.074373 : 1.0 : 0 : 18
2024-10-01 13:34:50.549729 : 1.0 : 0 : 19
2024-10-01 13:35:00.016218 : 1.0 : 0 : 20
2024-10-01 13:35:09.467131 : 1.0 : 0 : 21
2024-10-01 13:35:19.029195 : 1.0 : 0 : 22
2024-10-01 13:35:28.525138 : 1.0 : 0 : 23
2024-10-01 13:35:38.030214 : 1.0 : 0 : 24
2024-10-01 13:35:47.616299 : 1.0 : 0 : 25
2024-10-01 13:35:57.093345 : 1.0 : 0 : 26
2024-10-01 13:36:06.590964 : 1.0 : 0 : 27
2024-10-01 13:36:16.098848 : 1.0 : 0 : 

,instruction,output,input,llama2_fn_rag_s_1.0,llama2_fn_rag_s_0.9,llama2_fn_rag_s_0.8,llama2_fn_rag_s_0.7,llama2_fn_rag_s_0.6,llama2_fn_rag_s_0.5,llama2_fn_rag_s_0.4,llama2_fn_rag_s_0.3,llama2_fn_rag_s_0.2,llama2_fn_rag_s_0.1,llama2_fn_rag_s_0.0
0,What is (are) actin-accumulation myopathy ?,Actin-accumulation myopathy is a disorder that...,MedQuAD Super Expert,Actin-accumulation myopathy is a disorder of s...,Actin-accumulation myopathy is a disorder of s...,How many people are affected by actin-accumula...,Actin-accumulation myopathy can be inherited e...,What causes myosin storage myopathy? The exact...,The prevention of myoclonus requires physical ...,"Congenital myopathy of the skeletal muscles, i...",Congenital myosin storage myopathy: characteri...,The treatment depends on the specific type of ...,Congenital myosin storage disease: characteriz...,The three main types of myosin storage myopath...
1,Is Hydrocephalus due to congenital stenosis of...,Is hydrocephalus due to congenital stenosis of...,MedQuAD Super Expert,This condition is inherited in an autosomal re...,Is genetic testing available for hydrocephalus...,These resources address the diagnosis or manag...,Congenital hydrocephalus may occur alone or it...,Signs and symptoms vary depending on the size ...,The signs and symptoms of hydrocephalus due to...,For those with hydrocephalus due to congenital...,The cause of acquired hydrocephalus is often u...,The exact cause of congenital hydrocephalus is...,The cause of acquired hydrocephalus is unknown...,The cause of acquired hydrocephalus remains un...
2,What are the symptoms of Laing distal myopathy ?,What are the signs and symptoms of Laing dista...,MedQuAD Super Expert,What are the signs and symptoms of Laing dista...,What are the signs and symptoms of Laing dista...,What are the signs and symptoms of Flexion con...,What are the signs and symptoms of Lambert-Eat...,"Many of these symptoms apply to some, but not ...",Signs and symptoms vary depending on the speci...,The Human Phenotype Ontology provides the foll...,The Human Phenotype Ontology provides the foll...,The Human Phenotype Ontology provides the foll...,What are the signs and symptoms of Dandy-Walke...,Signs and symptoms vary from severe life-threa...
3,How many people are affected by hereditary hyp...,X-linked hypophosphatemic rickets is the most ...,MedQuAD Super Expert,Hereditary hypophosphatemic rickets affects ab...,Hereditary hypophosphatemic rickets affects ab...,This condition is usually inherited in an X-li...,A health care provider will determine how seve...,Health care providers recommend that children ...,What causes rickets? The exact underlying caus...,Familial or genetic hypolipoproteinemia (FGH) ...,Familial or genetic hypolipoproteinemia causes...,What causes familial or genetic hypolipoprotei...,Familial or genetic hypolipoproteinemia (FGH) ...,Familial or genetic hypolipoproteinemia causes...
4,What are the genetic changes related to Li-Fra...,The CHEK2 and TP53 genes are associated with L...,MedQuAD Super Expert,Mutations in the TP53 gene cause most cases of...,Mutations in at least four genes can cause Li-...,Mutations in the TP53 gene cause most cases of...,The following resources may be helpful as you ...,The cause of some cases of Li-Fraumeni syndrom...,Most cases of Pierson syndrome and some cases ...,"Most cases of Li-Fraumeni syndrome, as well as...","Most cases of Pierson syndrome, as well as tho...",Most cases of Li-Fraumeni syndrome are not ass...,"Most cases of Pierson syndrome, and some cases...","The genes that cause Li-Fraumeni syndrome, Tur..."


#### Question based

In [ ]:
test_topic = pd.read_json(f"{dir_root}qaa_test.json", orient="records")
eb = ExtractorBase(path = dir_root, domain_expert = domain_expert)
print(eb.embedded_vectors.shape)
test_topic = rag_test(domain_expert, eb, test_topic, "llama2_fn_rag_q_")
test_topic.to_json(f"{dir_root}/qaa_test_llama2_fn_q.json", orient="records")
test_topic.head()

(12815, 384)
2024-10-01 22:36:14.149418 : 1.0 : 0 : 0
2024-10-01 22:36:23.502455 : 1.0 : 1 : 1
2024-10-01 22:36:32.915540 : 1.0 : 1 : 2
2024-10-01 22:36:42.283635 : 1.0 : 0 : 3
2024-10-01 22:36:51.611873 : 1.0 : 0 : 4
2024-10-01 22:37:01.014138 : 1.0 : 1 : 5
2024-10-01 22:37:10.460999 : 1.0 : 2 : 6
2024-10-01 22:37:19.821814 : 1.0 : 1 : 7
2024-10-01 22:37:29.326591 : 1.0 : 0 : 8
2024-10-01 22:37:38.699120 : 1.0 : 2 : 9
2024-10-01 22:37:48.221799 : 1.0 : 1 : 10
2024-10-01 22:37:57.633892 : 1.0 : 1 : 11
2024-10-01 22:38:07.053849 : 1.0 : 1 : 12
2024-10-01 22:38:16.420305 : 1.0 : 2 : 13
2024-10-01 22:38:25.795524 : 1.0 : 1 : 14
2024-10-01 22:38:35.177803 : 1.0 : 0 : 15
2024-10-01 22:38:44.597372 : 1.0 : 2 : 16
2024-10-01 22:38:53.983541 : 1.0 : 1 : 17
2024-10-01 22:39:03.320806 : 1.0 : 0 : 18
2024-10-01 22:39:12.727671 : 1.0 : 1 : 19
2024-10-01 22:39:22.156226 : 1.0 : 1 : 20
2024-10-01 22:39:31.602973 : 1.0 : 3 : 21
2024-10-01 22:39:40.947915 : 1.0 : 0 : 22
2024-10-01 22:39:50.287169 : 1.

/tmp/ipykernel_18777/2825354160.py:51: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  self.duplicated_indexes = self.duplicated_indexes.groupby(list(self.duplicated_indexes)).apply(lambda x: x.index[0]).tolist()


2024-10-01 23:11:17.051704 : 1.0 : 16 : 222
2024-10-01 23:11:26.541314 : 1.0 : 0 : 223
2024-10-01 23:11:35.976363 : 1.0 : 1 : 224
2024-10-01 23:11:45.445873 : 1.0 : 1 : 225
2024-10-01 23:11:54.904561 : 1.0 : 1 : 226
2024-10-01 23:12:04.282576 : 1.0 : 0 : 227
2024-10-01 23:12:13.871940 : 1.0 : 2 : 228
2024-10-01 23:12:23.227783 : 1.0 : 2 : 229
2024-10-01 23:12:32.879522 : 1.0 : 0 : 230
2024-10-01 23:12:42.408848 : 1.0 : 0 : 231
2024-10-01 23:12:51.849367 : 1.0 : 1 : 232
2024-10-01 23:13:01.323276 : 1.0 : 0 : 233
2024-10-01 23:13:10.746702 : 1.0 : 0 : 234
2024-10-01 23:13:20.162040 : 1.0 : 3 : 235
2024-10-01 23:13:29.731530 : 1.0 : 1 : 236
2024-10-01 23:13:39.144099 : 1.0 : 2 : 237
2024-10-01 23:13:48.461363 : 1.0 : 1 : 238
2024-10-01 23:13:58.118208 : 1.0 : 0 : 239
2024-10-01 23:14:07.501105 : 1.0 : 1 : 240
2024-10-01 23:14:16.807130 : 1.0 : 0 : 241
2024-10-01 23:14:26.122406 : 1.0 : 0 : 242
2024-10-01 23:14:35.450229 : 1.0 : 1 : 243
2024-10-01 23:14:44.801454 : 1.0 : 1 : 244
2024-10-01

/tmp/ipykernel_18777/2825354160.py:51: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  self.duplicated_indexes = self.duplicated_indexes.groupby(list(self.duplicated_indexes)).apply(lambda x: x.index[0]).tolist()


2024-10-01 23:20:52.447336 : 1.0 : 1 : 283
2024-10-01 23:21:01.824268 : 1.0 : 0 : 284
2024-10-01 23:21:11.287132 : 1.0 : 1 : 285
2024-10-01 23:21:20.706598 : 1.0 : 2 : 286
2024-10-01 23:21:30.205695 : 1.0 : 0 : 287
2024-10-01 23:21:39.698490 : 1.0 : 0 : 288
2024-10-01 23:21:49.150348 : 1.0 : 1 : 289
2024-10-01 23:21:58.584274 : 1.0 : 1 : 290
2024-10-01 23:22:08.062184 : 1.0 : 0 : 291
2024-10-01 23:22:17.430394 : 1.0 : 1 : 292
2024-10-01 23:22:26.770999 : 1.0 : 0 : 293
2024-10-01 23:22:36.302590 : 1.0 : 0 : 294
2024-10-01 23:22:45.660892 : 1.0 : 0 : 295
2024-10-01 23:22:55.086077 : 1.0 : 2 : 296
2024-10-01 23:23:04.679200 : 1.0 : 0 : 297
2024-10-01 23:23:14.061922 : 1.0 : 0 : 298
2024-10-01 23:23:23.518463 : 1.0 : 1 : 299
2024-10-01 23:23:33.037426 : 0.9 : 1 : 0
2024-10-01 23:23:42.567175 : 0.9 : 2 : 1
2024-10-01 23:23:51.992556 : 0.9 : 1 : 2
2024-10-01 23:24:01.307200 : 0.9 : 1 : 3
2024-10-01 23:24:10.853131 : 0.9 : 1 : 4
2024-10-01 23:24:20.309608 : 0.9 : 1 : 5
2024-10-01 23:24:29.844

...

2024-10-02 02:03:29.446741 : 0.6 : 77 : 101


/tmp/ipykernel_18777/2825354160.py:51: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  self.duplicated_indexes = self.duplicated_indexes.groupby(list(self.duplicated_indexes)).apply(lambda x: x.index[0]).tolist()


2024-10-02 02:03:39.319397 : 0.6 : 78 : 102
2024-10-02 02:03:48.933873 : 0.6 : 8 : 103
2024-10-02 02:03:58.692678 : 0.6 : 100 : 104


/tmp/ipykernel_18777/2825354160.py:51: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  self.duplicated_indexes = self.duplicated_indexes.groupby(list(self.duplicated_indexes)).apply(lambda x: x.index[0]).tolist()


2024-10-02 02:04:08.511199 : 0.6 : 68 : 105
2024-10-02 02:04:18.098199 : 0.6 : 9 : 106
2024-10-02 02:04:27.799803 : 0.6 : 100 : 107
2024-10-02 02:04:37.494219 : 0.6 : 99 : 108
2024-10-02 02:04:47.098509 : 0.6 : 31 : 109
2024-10-02 02:04:56.624239 : 0.6 : 47 : 110
2024-10-02 02:05:06.281912 : 0.6 : 85 : 111


In [ ]:
domain_expert.model_st = None
domain_expert.pipeline_chat = None
domain_expert.tokenizer_chat = None
domain_expert = None
eb = None
ebf = None
test_topic = None
gc.collect()
torch.cuda.empty_cache()

### LLama2 RAG with Context FN model

In [58]:
domain_expert = DomainExpertV2(min_new_tokens=min_token, 
                               max_new_tokens=max_token,
                               type=type,
                               model_chat_name="Models/llama-2-7b-hf-context-v20240927/checkpoint-960")

/opt/conda/envs/domaine/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

#### Sentecen based

In [59]:
test_topic = pd.read_json(f"{dir_root}qaa_test.json", orient="records")
ebf = ExtractorBaseF(path = dir_root, domain_expert = domain_expert)
print(ebf.embedded_vectors.shape)
test_topic = rag_test(domain_expert, ebf, test_topic, mark = "llama2c_rag_s_", N=1000)
test_topic.to_json(f"{dir_root}/qaa_test_llama2c_s.json", orient="records")
test_topic.head()

(130777, 384)
2024-09-29 15:45:32.915591 : 1.0 : 0 : 0
2024-09-29 15:45:42.766123 : 1.0 : 1 : 1
2024-09-29 15:45:52.250606 : 1.0 : 0 : 2
2024-09-29 15:46:01.710551 : 1.0 : 0 : 3
2024-09-29 15:46:11.344724 : 1.0 : 0 : 4
2024-09-29 15:46:21.113697 : 1.0 : 0 : 5
2024-09-29 15:46:30.575430 : 1.0 : 0 : 6
2024-09-29 15:46:40.028344 : 1.0 : 0 : 7
2024-09-29 15:46:49.498471 : 1.0 : 0 : 8
2024-09-29 15:46:59.027919 : 1.0 : 0 : 9
2024-09-29 15:47:08.586163 : 1.0 : 0 : 10
2024-09-29 15:47:18.132362 : 1.0 : 0 : 11
2024-09-29 15:47:27.693412 : 1.0 : 0 : 12
2024-09-29 15:47:37.392788 : 1.0 : 0 : 13
2024-09-29 15:47:47.374372 : 1.0 : 0 : 14
2024-09-29 15:47:57.222471 : 1.0 : 0 : 15
2024-09-29 15:48:07.073856 : 1.0 : 0 : 16
2024-09-29 15:48:16.661413 : 1.0 : 0 : 17
2024-09-29 15:48:26.552660 : 1.0 : 0 : 18
2024-09-29 15:48:36.486786 : 1.0 : 0 : 19
2024-09-29 15:48:46.198557 : 1.0 : 0 : 20
2024-09-29 15:48:56.238473 : 1.0 : 0 : 21
2024-09-29 15:49:06.191024 : 1.0 : 0 : 22
2024-09-29 15:49:16.082872 : 1

,instruction,output,input,llama2c_rag_s_1.0,llama2c_rag_s_0.9,llama2c_rag_s_0.8,llama2c_rag_s_0.7,llama2c_rag_s_0.6,llama2c_rag_s_0.5,llama2c_rag_s_0.4,llama2c_rag_s_0.3,llama2c_rag_s_0.2,llama2c_rag_s_0.1,llama2c_rag_s_0.0
0,What is (are) actin-accumulation myopathy ?,Actin-accumulation myopathy is a disorder that...,MedQuAD Super Expert,Actin-accumulation myopathy is a disorder that...,Actin-accumulation myopathy is a disorder that...,Actin-accumulation myopathy can be inherited a...,Actin-accumulation myopathy is most often inhe...,What causes inclusion body myopathy 2? Inclusi...,The prognosis for individuals with the myopath...,The prognosis for persons with tubular aggrega...,The prognosis for individuals with tubular agg...,Neurological manifestations of the myopathies ...,The prognosis for individuals with the myopath...,The prognosis for individuals with tubular agg...
1,Is Hydrocephalus due to congenital stenosis of...,Is hydrocephalus due to congenital stenosis of...,MedQuAD Super Expert,When hydrocephalus due to congenital stenosis ...,Diagnosis is typically made based on the prese...,These resources address the diagnosis or manag...,These resources address the diagnosis or manag...,Diagnosis is typically made based on symptoms ...,Although the cause of ps I is not fully unders...,Although babies with HD do not usually show an...,The signs and symptoms of HS include the follo...,Although many children with hydrocephalus do n...,There are four types of hydrocephalus that are...,"The signs and symptoms of HSAN2 include ""dubby..."
2,What are the symptoms of Laing distal myopathy ?,What are the signs and symptoms of Laing dista...,MedQuAD Super Expert,What are the signs and symptoms of Laing dista...,What are the signs and symptoms of Laing dista...,Bethlem myopathy mainly affects skeletal muscl...,What are the signs and symptoms of Laing dista...,"Medications may worsen some symptoms, such as ...",The signs and symptoms of the disease may appe...,The signs and symptoms of the disease may appe...,The signs and symptoms of MMA usually progress...,The signs and symptoms of MPS I and MPS II ove...,Major Signs and Symptoms\n \nSi...,Most individuals with myopathy have mild to mo...
3,How many people are affected by hereditary hyp...,X-linked hypophosphatemic rickets is the most ...,MedQuAD Super Expert,The exact prevalence of hereditary hypophospha...,The exact prevalence of hereditary hypophospha...,This condition is inherited in an autosomal re...,A health care provider treats hypophosphatemic...,The Human Phenotype Ontology (HPO) has collect...,Hypophosphatasia is almost always inherited in...,Hypophosphatasia is almost always inherited in...,Hypophosphatasia is almost always inherited in...,Familial isolated hyperparathyroidism (FIHP) i...,This condition is inherited in an autosomal do...,Familial isolated hyperparathyroidism (FIHP) i...
4,What are the genetic changes related to Li-Fra...,The CHEK2 and TP53 genes are associated with L...,MedQuAD Super Expert,Li-Fraumeni syndrome is caused by mutations in...,Mutations in the TWIST1 gene cause Li-Fraumeni...,Li-Fraumeni syndrome is caused by mutations in...,Key Points\n - There are di...,This condition is inherited in an autosomal re...,Most cases of Li-Fraumeni syndrome and a small...,Most cases of Li-Fraumeni syndrome and a small...,Most cases of Li-Fraumeni syndrome and a small...,Majeed syndrome results from mutations in the ...,This condition is inherited in an autosomal re...,"Li-Fraumeni syndrome, Turcot syndrome, and tub..."


#### Question based

In [61]:
test_topic = pd.read_json(f"{dir_root}qaa_test.json", orient="records")
eb = ExtractorBase(path = dir_root, domain_expert = domain_expert)
print(eb.embedded_vectors.shape)
test_topic = rag_test(domain_expert, eb, test_topic, "llama2c_rag_q_")
test_topic.to_json(f"{dir_root}/qaa_test_llama2c_q.json", orient="records")
test_topic.head()

(12815, 384)
2024-09-30 07:40:07.140118 : 1.0 : 0 : 0
2024-09-30 07:40:16.853957 : 1.0 : 1 : 1
2024-09-30 07:40:26.529476 : 1.0 : 1 : 2
2024-09-30 07:40:36.060220 : 1.0 : 0 : 3
2024-09-30 07:40:45.475545 : 1.0 : 0 : 4
2024-09-30 07:40:54.979963 : 1.0 : 1 : 5
2024-09-30 07:41:04.465256 : 1.0 : 2 : 6
2024-09-30 07:41:13.915220 : 1.0 : 1 : 7
2024-09-30 07:41:23.370446 : 1.0 : 0 : 8
2024-09-30 07:41:32.998706 : 1.0 : 2 : 9
2024-09-30 07:41:42.595699 : 1.0 : 1 : 10
2024-09-30 07:41:52.035829 : 1.0 : 1 : 11
2024-09-30 07:42:01.546671 : 1.0 : 1 : 12
2024-09-30 07:42:10.956557 : 1.0 : 2 : 13
2024-09-30 07:42:20.583551 : 1.0 : 1 : 14
2024-09-30 07:42:30.156703 : 1.0 : 0 : 15
2024-09-30 07:42:39.632602 : 1.0 : 2 : 16
2024-09-30 07:42:49.084483 : 1.0 : 1 : 17
2024-09-30 07:42:58.500660 : 1.0 : 0 : 18
2024-09-30 07:43:08.054025 : 1.0 : 1 : 19
2024-09-30 07:43:17.489862 : 1.0 : 1 : 20
2024-09-30 07:43:27.167814 : 1.0 : 3 : 21
2024-09-30 07:43:36.653194 : 1.0 : 0 : 22
2024-09-30 07:43:46.240071 : 1.

/tmp/ipykernel_554/2825354160.py:51: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  self.duplicated_indexes = self.duplicated_indexes.groupby(list(self.duplicated_indexes)).apply(lambda x: x.index[0]).tolist()


2024-09-30 08:15:12.669251 : 1.0 : 16 : 222
2024-09-30 08:15:22.047611 : 1.0 : 0 : 223
2024-09-30 08:15:31.508403 : 1.0 : 1 : 224
2024-09-30 08:15:40.960547 : 1.0 : 1 : 225
2024-09-30 08:15:50.580832 : 1.0 : 1 : 226
2024-09-30 08:15:59.964630 : 1.0 : 0 : 227
2024-09-30 08:16:09.634827 : 1.0 : 2 : 228
2024-09-30 08:16:19.034608 : 1.0 : 2 : 229
2024-09-30 08:16:28.503494 : 1.0 : 0 : 230
2024-09-30 08:16:37.974026 : 1.0 : 0 : 231
2024-09-30 08:16:47.436572 : 1.0 : 1 : 232
2024-09-30 08:16:56.886359 : 1.0 : 0 : 233
2024-09-30 08:17:06.355202 : 1.0 : 0 : 234
2024-09-30 08:17:15.897873 : 1.0 : 3 : 235
2024-09-30 08:17:25.467076 : 1.0 : 1 : 236
2024-09-30 08:17:34.882414 : 1.0 : 2 : 237
2024-09-30 08:17:44.360451 : 1.0 : 1 : 238
2024-09-30 08:17:53.780271 : 1.0 : 0 : 239
2024-09-30 08:18:03.263560 : 1.0 : 1 : 240
2024-09-30 08:18:12.708528 : 1.0 : 0 : 241
2024-09-30 08:18:22.072171 : 1.0 : 0 : 242
2024-09-30 08:18:31.454077 : 1.0 : 1 : 243
2024-09-30 08:18:40.909730 : 1.0 : 1 : 244
2024-09-30

/tmp/ipykernel_554/2825354160.py:51: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  self.duplicated_indexes = self.duplicated_indexes.groupby(list(self.duplicated_indexes)).apply(lambda x: x.index[0]).tolist()


2024-09-30 08:24:49.121019 : 1.0 : 1 : 283
2024-09-30 08:24:58.741317 : 1.0 : 0 : 284
2024-09-30 08:25:08.157725 : 1.0 : 1 : 285
2024-09-30 08:25:17.665034 : 1.0 : 2 : 286
2024-09-30 08:25:27.041225 : 1.0 : 0 : 287
2024-09-30 08:25:36.579039 : 1.0 : 0 : 288
2024-09-30 08:25:46.090738 : 1.0 : 1 : 289
2024-09-30 08:25:55.556011 : 1.0 : 1 : 290
2024-09-30 08:26:04.991669 : 1.0 : 0 : 291
2024-09-30 08:26:14.461448 : 1.0 : 1 : 292
2024-09-30 08:26:23.848516 : 1.0 : 0 : 293
2024-09-30 08:26:33.219845 : 1.0 : 0 : 294
2024-09-30 08:26:42.638811 : 1.0 : 0 : 295
2024-09-30 08:26:52.109331 : 1.0 : 2 : 296
2024-09-30 08:27:01.612496 : 1.0 : 0 : 297
2024-09-30 08:27:10.991889 : 1.0 : 0 : 298
2024-09-30 08:27:20.361760 : 1.0 : 1 : 299
2024-09-30 08:27:29.824280 : 0.9 : 1 : 0
2024-09-30 08:27:39.338997 : 0.9 : 2 : 1
2024-09-30 08:27:48.743617 : 0.9 : 1 : 2
2024-09-30 08:27:58.124229 : 0.9 : 1 : 3
2024-09-30 08:28:07.600359 : 0.9 : 1 : 4
2024-09-30 08:28:17.026007 : 0.9 : 1 : 5
2024-09-30 08:28:26.455

...

/tmp/ipykernel_554/2825354160.py:51: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  self.duplicated_indexes = self.duplicated_indexes.groupby(list(self.duplicated_indexes)).apply(lambda x: x.index[0]).tolist()


2024-09-30 16:34:35.005747 : 0.0 : 98 : 297
2024-09-30 16:34:44.898398 : 0.0 : 100 : 298


/tmp/ipykernel_554/2825354160.py:51: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  self.duplicated_indexes = self.duplicated_indexes.groupby(list(self.duplicated_indexes)).apply(lambda x: x.index[0]).tolist()


2024-09-30 16:34:54.842334 : 0.0 : 99 : 299


,instruction,output,input,llama2c_rag_q_1.0,llama2c_rag_q_0.9,llama2c_rag_q_0.8,llama2c_rag_q_0.7,llama2c_rag_q_0.6,llama2c_rag_q_0.5,llama2c_rag_q_0.4,llama2c_rag_q_0.3,llama2c_rag_q_0.2,llama2c_rag_q_0.1,llama2c_rag_q_0.0
0,What is (are) actin-accumulation myopathy ?,Actin-accumulation myopathy is a disorder that...,MedQuAD Super Expert,Actin-accumulation myopathy is a disorder that...,Actin-accumulation myopathy is usually inherit...,Actin-accumulation myopathy is usually inherit...,Myosin storage myopathy is caused by mutations...,Type I tubular aggregate myopathy is a slowly ...,Type I fibers are part of a group of fibers ca...,Type I fibers are part of a protein complex ca...,Type I and type II fibers make up the two type...,Type I fibers make up the majority of muscles ...,Type I fibers are normally made up of three an...,Type I tubular aggregate myopathy is caused by...
1,Is Hydrocephalus due to congenital stenosis of...,Is hydrocephalus due to congenital stenosis of...,MedQuAD Super Expert,A female with an X-linked recessive condition ...,How might hydrocephalus due to congenital sten...,HSAS is typically treated with surgical remova...,These resources address the diagnosis or manag...,"In most cases, an affected person has one pare...","In many cases, RAS has no symptoms until it be...",When Cornelia de Lange syndrome is caused by m...,"In most cases, an affected person has one pare...",When the condition is caused by a mutation in ...,"In most cases, an affected person has one pare...",When the condition is caused by a mutation in ...
2,What are the symptoms of Laing distal myopathy ?,What are the signs and symptoms of Laing dista...,MedQuAD Super Expert,The prognosis for individuals with HSP depends...,The prevalence of Laing distal myopathy is unk...,The HPO has identified 29 cases of tubular agg...,The genetic cause of Laing distal myopathy is ...,The prognosis for individuals with Laing dista...,The genetic causes of Laing distal myopathy ar...,Weakness in the tibialis anterior alters the w...,The prognosis for individuals with Laing dista...,The Human Phenotype Ontology (HPO) has collect...,"The genes involved in Laing distal myopathy, i...",The Human Phenotype Ontology (HPO) has collect...
3,How many people are affected by hereditary hyp...,X-linked hypophosphatemic rickets is the most ...,MedQuAD Super Expert,The exact incidence of this condition is unkno...,The exact prevalence of X-linked hypophosphate...,This condition is inherited in an X-linked rec...,The Human Phenotype Ontology (HPO) has collect...,The milder forms of hypophosphatasia are much ...,The prevalence of the autosomal recessive form...,The prognosis for persons with hypophosphatasi...,The milder forms of hypophosphatasia are much ...,The most common signs and symptoms of hypophos...,The milder forms of hypophosphatasia that appe...,The most common signs and symptoms of heart fa...
4,What are the genetic changes related to Li-Fra...,The CHEK2 and TP53 genes are associated with L...,MedQuAD Super Expert,Mutations in the TWIST1 gene cause Li-Fraumeni...,Mutations in the CHEK2 gene increase the risk ...,About 1 in 7 men will be diagnosed with prosta...,Bowen-Conradi syndrome is usually caused by a ...,"Unlike most other cancers, leukemia isn't a tu...",Bowen-Conradi syndrome is usually caused by a ...,Bowen-Conradi syndrome is usually caused by a ...,We are aware of only one EMG1 gene mutation th...,Bowen-Conradi syndrome is usually caused by a ...,"Unlike most other cancers, leukemia isn't a tu...",Bowen-Conradi syndrome is usually caused by a ...


In [62]:
domain_expert.model_st = None
domain_expert.pipeline_chat = None
domain_expert.tokenizer_chat = None
domain_expert = None
eb = None
ebf = None
test_topic = None
gc.collect()
torch.cuda.empty_cache()

## Concat

In [27]:
test_topic = pd.read_json(f"{dir_root}qaa_test.json", orient="records")
test_topic_llama2_base_s = pd.read_json(f"{dir_root}qaa_test_llama2_base_s.json", orient="records")
test_topic_llama2_base_q = pd.read_json(f"{dir_root}qaa_test_llama2_base_q.json", orient="records")
test_topic_llama2_fn_s = pd.read_json(f"{dir_root}qaa_test_llama2_fn_s.json", orient="records")
test_topic_llama2_fn_q = pd.read_json(f"{dir_root}qaa_test_llama2_fn_q.json", orient="records")
test_topic_llama2c_s = pd.read_json(f"{dir_root}qaa_test_llama2c_s.json", orient="records")
test_topic_llama2c_q = pd.read_json(f"{dir_root}qaa_test_llama2c_q.json", orient="records")

test_topic_gptj_fn = pd.read_json(f"{dir_root}/qaa_test_gptj_fn.json", orient="records")
test_topic_opt_fn = pd.read_json(f"{dir_root}/qaa_test_opt_fn.json", orient="records")
test_topic_llama_fn = pd.read_json(f"{dir_root}/qaa_test_llama_fn.json", orient="records")
test_topic_llama2_fn = pd.read_json(f"{dir_root}/qaa_test_llama2_fn.json", orient="records")
test_topic_llama2_base = pd.read_json(f"{dir_root}/qaa_test_llama2_base.json", orient="records")

test_topic["gptj_fn"] = test_topic_gptj_fn["gptj_fn"]
test_topic["opt_fn"] = test_topic_opt_fn["opt_fn"]
test_topic["llama_fn"] = test_topic_llama_fn["llama_fn"]
test_topic["llama2_fn"] = test_topic_llama2_fn["llama2_fn"]
test_topic["llama2_base"] = test_topic_llama2_base["llama2_base"]

for r in ["llama2_rag_s_" + str(round(treshold,1)) for treshold in np.arange(1.0, -0.1, -0.1)]:
    test_topic[r] = test_topic_llama2_base_s[r]

for r in ["llama2_rag_q_" + str(round(treshold,1)) for treshold in np.arange(1.0, -0.1, -0.1)]:    
    test_topic[r] = test_topic_llama2_base_q[r]

for r in ["llama2_fn_rag_s_" + str(round(treshold,1)) for treshold in np.arange(1.0, -0.1, -0.1)]:
    test_topic[r] = test_topic_llama2_fn_s[r]

for r in ["llama2_fn_rag_q_" + str(round(treshold,1)) for treshold in np.arange(1.0, -0.1, -0.1)]:
    test_topic[r] = test_topic_llama2_fn_q[r]
    
for r in ["llama2c_rag_s_" + str(round(treshold,1)) for treshold in np.arange(1.0, -0.1, -0.1)]:        
    test_topic[r] = test_topic_llama2c_s[r]

for r in ["llama2c_rag_q_" + str(round(treshold,1)) for treshold in np.arange(1.0, -0.1, -0.1)]:    
    test_topic[r] = test_topic_llama2c_q[r]

test_topic.to_json(f"{dir_root}/qaa_test_measured.json", orient="records")

In [30]:
from IPython import get_ipython
get_ipython().kernel.do_shutdown(restart=True)

{'status': 'ok', 'restart': True}